# Data Loading, Cleaning & Export

### Importazione Librerie

In [27]:
import numpy as np
import pandas as pd
import requests

### Caricamento Dati

In [38]:
CSV_files = {
    "customers":           r"dataset/olist_customers_dataset.csv",
    "geolocation_dataset": r"dataset/olist_geolocation_dataset.csv",
    "order_items":         r"dataset/olist_order_items_dataset.csv",
    "order_payments":      r"dataset/olist_order_payments_dataset.csv",
    "order_review":        r"dataset/olist_order_reviews_dataset.csv",
    "order_dataset":       r"dataset/olist_orders_dataset.csv",
    "list_product":        r"dataset/olist_products_dataset.csv",
    "list_seller":         r"dataset/olist_sellers_dataset.csv",
    "product_category":    r"dataset/product_category_name_translation.csv"
}

dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

df_customers           = dataframes["customers"]
df_orders_items        = dataframes["order_items"]
df_order_payments      = dataframes["order_payments"]
df_order_review        = dataframes["order_review"]
df_order_dataset       = dataframes["order_dataset"]
df_list_product        = dataframes["list_product"]
df_list_seller         = dataframes["list_seller"]
df_product_category    = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

### Analisi Preliminare

In [3]:
# Stampa un riepilogo di ogni csv con la sua shape, i missing values e i duplicati.
def quick_overview(df, name):

    # Nome DF
    print(f"{name}")
    print(f"Shape: {df.shape}")

    # Valori nulli
    nulls = df.isna().sum()
    nulls = nulls[nulls > 0]
    if len(nulls) > 0:
        print(f"Valori nulli:\n{nulls.to_string()}\n")
    else:
        print("Nessun valore nullo\n")

for name, df in dataframes.items():
    quick_overview(df, name)

customers
Shape: (99441, 5)
Nessun valore nullo

geolocation_dataset
Shape: (1000163, 5)
Nessun valore nullo

order_items
Shape: (112650, 7)
Nessun valore nullo

order_payments
Shape: (103886, 5)
Nessun valore nullo

order_review
Shape: (99224, 7)
Valori nulli:
review_comment_title      87656
review_comment_message    58247

order_dataset
Shape: (99441, 8)
Valori nulli:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965

list_product
Shape: (32951, 9)
Valori nulli:
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2

list_seller
Shape: (3095, 4)
Nessun valore nullo

product_category
Shape: (71, 2)
Nessun valore nullo



### PULIZIA: df_list_product

In [29]:
# 1) Droppiamo le colonne (peso, misure)
# 2) Droppiamo i 610 prodotti senza categoria (< 2% del totale):
#    verranno esclusi anche dagli altri df.
# 3) Uniamo la traduzione inglese della categoria.

# 1
df_list_product = df_list_product.drop(
    columns=['product_name_lenght', 'product_width_cm',
             'product_height_cm', 'product_length_cm', 'product_weight_g']
)

# 2
# Salviamo gli id dei prodotti senza categoria per filtrare order_items dopo
product_id_nan = df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete = product_id_nan['product_id'].to_numpy()
df_list_product = df_list_product.dropna(subset=['product_category_name'])

# 3
df_list_product = (
    df_list_product
    .merge(df_product_category, on='product_category_name', how='left')
)

df_list_product.info()
df_list_product.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  str    
 1   product_category_name          32341 non-null  str    
 2   product_description_lenght     32341 non-null  float64
 3   product_photos_qty             32341 non-null  float64
 4   product_category_name_english  32328 non-null  str    
dtypes: float64(2), str(3)
memory usage: 1.2 MB


,product_id,product_category_name,product_description_lenght,product_photos_qty,product_category_name_english
13535,5a9390e32c656cca81df71054e77b93b,automotivo,647.0,1.0,auto
5558,3ae810052cf18d30232367f0b45e284c,alimentos_bebidas,371.0,2.0,food_drink


In [30]:
# Traduciamo con Selenium le categorie portoghesi senza traduzione inglese
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Troviamo i nomi portoghesi che non hanno traduzione nel dataframe df_product_category
traduzione_mancante = df_list_product['product_category_name_english'].isna()
nomi_mancanti = df_list_product[traduzione_mancante]['product_category_name'].unique()
print(f"Categorie da tradurre ({len(nomi_mancanti)}): \n{nomi_mancanti}\n")


ModuleNotFoundError: No module named 'selenium'

In [31]:
# Apre Google Translate nel browser e restituisce la traduzione
# dal portoghese all'inglese del testo passato, formattata in snake_case.
def traduci_categoria(testo):

    # Costruisce l'URL di Google Translate con i parametri: lingua portoghese (pt), lingua inglese (en) e testo
    url = f"https://translate.google.com/?sl=pt&tl=en&text={testo}&op=translate"
    driver.get(url)
    
    # Attende 2 secondi che la pagina carichi e la traduzione appaia
    time.sleep(2)

    # Accetta i cookie se il popup è presente
    try:
        accept_button = driver.find_element(By.XPATH, "//button[.//span[text()='Accept all']]")
        accept_button.click()
        time.sleep(2)
    except:
        pass  # Il popup non è presente, continua normalmente
    
    # Trova l'elemento HTML che contiene il testo tradotto
    risultato = driver.find_element(By.XPATH, "//span[@jsname='W297wb']")
    
    # Converte in minuscolo e sostituisce gli spazi con underscore
    traduzione = risultato.text.lower().replace(' ', '_')
    return traduzione

# Avvia Firefox
# NB CONTROLLATE TUTTI SE SI APRE IL BROWSER E VEDETE GOOGLE TRANSLATE
driver = webdriver.Firefox()

# Dizionario che conterrà le traduzioni: {nome_portoghese: traduzione_inglese}
traduzioni_manuali = {}

# Itera sui nomi di categoria mancanti di traduzione
for nome in nomi_mancanti:
    traduzione = traduci_categoria(nome)
    traduzioni_manuali[nome] = traduzione
    print(f"{nome}  →  {traduzione}")
    time.sleep(1)

# Chiude il browser al termine delle traduzioni
driver.quit()

# Aggiorna il DataFrame: per ogni categoria tradotta,
# riempie i valori NaN nella colonna 'product_category_name_english'
for nome, traduzione in traduzioni_manuali.items():
    df_list_product.loc[
        df_list_product['product_category_name'] == nome,
        'product_category_name_english'
    ] = traduzione

df_list_product.info()

NameError: name 'webdriver' is not defined

### PULIZIA: df_orders_items

In [32]:
url = 'https://v6.exchangerate-api.com/v6/ee8e65018f5adf36f58283bd/latest/EUR'
oae = requests.get(url)
print(oae.status_code)#richiesta API  

200


In [33]:
EUR_BRL=oae.json()['conversion_rates']['BRL']#Stanziata la variabile del tasso di cambio chiamndolo EUR_BRL
EUR_BRL#tasso conversione

6.1292

In [34]:
# 1) Escludiamo gli items con product_id in product_id_to_delete
# 2) Convertiamo shipping_limit_date in datetime

# 1
df_orders_items = (
    df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
    .copy()
    .reset_index(drop=True)
)

# 2
df_orders_items['shipping_limit_date'] = pd.to_datetime(
    df_orders_items['shipping_limit_date']
)

#conversione in euro di price e freight_value
df_orders_items['eur_price']=round(df_orders_items['price']/EUR_BRL,2)
df_orders_items['eur_freight_value']=round(df_orders_items['freight_value']/EUR_BRL,2)
#rimozione colonne in real breasiliano
df_orders_items=df_orders_items.drop(['price','freight_value'],axis=1)


df_orders_items.info()
df_orders_items.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 111047 entries, 0 to 111046
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             111047 non-null  str           
 1   order_item_id        111047 non-null  int64         
 2   product_id           111047 non-null  str           
 3   seller_id            111047 non-null  str           
 4   shipping_limit_date  111047 non-null  datetime64[us]
 5   eur_price            111047 non-null  float64       
 6   eur_freight_value    111047 non-null  float64       
dtypes: datetime64[us](1), float64(2), int64(1), str(3)
memory usage: 5.9 MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,eur_price,eur_freight_value
2011,04993613aee4046caf92ea17b316dcfb,8,96d1c06692244e2fed6e1f9aa9552770,00ee68308b45bc5e2660cd833c3f81cc,2018-02-07 02:52:44,10.12,1.72
8858,148ba0f4f771be94ad368512ca50a07f,1,1ec486885049bbb9b79351d150ed18c4,cab85505710c7cb9b720bceb52b01cee,2018-03-14 13:05:33,8.14,2.97


### PULIZIA: df_order_dataset

In [35]:
# 1) Conversione delle colonne data in datetime

# 2) delivery_delay_days : differenza tra consegna effettiva e stimata
#    (positivo = in ritardo, negativo = in anticipo)
#    actual_delivery_days: giorni totali dall'acquisto alla consegna

# 1
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    df_order_dataset[col] = pd.to_datetime(df_order_dataset[col])

# 2
df_order_dataset['delivery_delay_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_estimated_delivery_date']
).dt.days
df_order_dataset['actual_delivery_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_purchase_timestamp']
).dt.days

df_order_dataset.info()
df_order_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  str           
 1   customer_id                    99441 non-null  str           
 2   order_status                   99441 non-null  str           
 3   order_purchase_timestamp       99441 non-null  datetime64[us]
 4   order_approved_at              99281 non-null  datetime64[us]
 5   order_delivered_carrier_date   97658 non-null  datetime64[us]
 6   order_delivered_customer_date  96476 non-null  datetime64[us]
 7   order_estimated_delivery_date  99441 non-null  datetime64[us]
 8   delivery_delay_days            96476 non-null  float64       
 9   actual_delivery_days           96476 non-null  float64       
dtypes: datetime64[us](5), float64(2), str(3)
memory usage: 7.6 MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,actual_delivery_days
57841,87a635f625b39db6169bd9ad1bc63925,de05e894924fb7aaf5fd64556425578a,delivered,2017-09-24 19:32:04,2017-09-24 19:45:30,2017-09-25 18:07:52,2017-10-04 21:37:30,2017-10-23,-19.0,10.0
56930,e53182bbd948b07916ff86a2b55978d7,463b9e51b7530105dbaf4649dd8edf99,delivered,2018-02-02 13:47:56,2018-02-02 14:51:49,2018-02-06 21:26:50,2018-02-16 15:25:02,2018-03-02,-14.0,14.0


### PULIZIA: df_order_review

In [36]:
# 1) Conversione della colonna review_creation_date in datetime
# 2) Droppiamo le colonne (review_comment_title, review_comment_message)

# 1
df_order_review['review_creation_date'] = pd.to_datetime(
    df_order_review['review_creation_date']
)

# 2
df_order_review = df_order_review.drop(columns= ['review_comment_title', 'review_comment_message'])

df_order_review.sample(2)

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
44671,719679b3a4d69edafeb252a1afc7ddcb,b2ec9793219a396874af11e39d08f183,1,2017-11-08,2017-11-13 20:26:42
2992,b4e7ca69d6debcdc6884813c12366510,8bb0e358b3866146681505153a180c4f,4,2017-06-27,2017-07-07 10:14:37


### PULIZIA: df_order_payments

In [42]:
# Per un ordine ci sono diverse righe di pagamenti
# Aggreghiamo per order_id

df_order_payments = (
    df_order_payments
    .groupby('order_id', as_index=False)
    .agg(
        total_payment_value= ('payment_value', 'sum'),
        payment_installments= ('payment_installments', 'max'),
        payment_type         = ('payment_type', 'first')
    )
)

df_order_payments.sample(2)

,order_id,total_payment_value,payment_installments,payment_type
84504,d94cd4d810e4cf28560cc3b12aa4b210,43.22,1,credit_card
77398,c72c794309b49932e42cf11c75b22c45,98.82,1,boleto


In [46]:
df_order_payments['eur_total_payment_value']=round(df_order_payments['total_payment_value']/EUR_BRL,2)#convesrione
df_order_payments=df_order_payments.drop(['total_payment_value'],axis=1)

In [47]:
df_order_payments.sample(2)

,order_id,payment_installments,payment_type,eur_total_payment_value
33072,5528ba50898b536ac11c799ba9eef5ac,8,credit_card,52.70
75563,c27c10569c789ec6293df7ef9f4caaa1,7,credit_card,18.94


### PULIZIA:df_geolocation

In [11]:
# Coordinate del Brasile
brazil = {
    "lat_min": -34.0, "lat_max": 5.0,
    "lng_min": -75.0, "lng_max": -28.0
}

# Filtra coordinate fuori dal Brasile
df_geolocation_clean = df_geolocation_dataset[
    df_geolocation_dataset["geolocation_lat"].between(brazil["lat_min"], brazil["lat_max"]) &
    df_geolocation_dataset["geolocation_lng"].between(brazil["lng_min"], brazil["lng_max"])
].copy()

# Raggruppa per zip code (media lat/lng, primo valore per città e stato)
df_geolocation_dataset = (
    df_geolocation_clean
    .groupby('geolocation_zip_code_prefix', as_index=False)
    .agg(
        geolocation_lat   = ('geolocation_lat',   'mean'),
        geolocation_lng   = ('geolocation_lng',   'mean'),
        geolocation_city  = ('geolocation_city',  'first'),
        geolocation_state = ('geolocation_state', 'first')
    )
)

df_geolocation_dataset.info()
df_geolocation_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 19011 entries, 0 to 19010
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19011 non-null  int64  
 1   geolocation_lat              19011 non-null  float64
 2   geolocation_lng              19011 non-null  float64
 3   geolocation_city             19011 non-null  str    
 4   geolocation_state            19011 non-null  str    
dtypes: float64(2), int64(1), str(2)
memory usage: 742.7 KB


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1167,3294,-23.592858,-46.536599,sao paulo,SP
11915,58188,-6.644907,-36.248173,barauna,PB


### PULIZIA: df_list_seller

In [12]:
df_list_seller=df_list_seller.rename(columns={'seller_zip_code_prefix':'geolocation_zip_code_prefix'})
df_list_seller.sample(3)

,seller_id,geolocation_zip_code_prefix,seller_city,seller_state
83,4e7c18b98d84e05cbae3ff0ff03846c2,14882,jaboticabal,SP
2675,23c38debaffe4a25a30fdbd9b586a13f,18110,votorantim,SP
2344,0e44d110fa6a54e121cb2c095a77762f,7901,francisco morato,SP


### PULIZIA: df_customers

In [13]:
df_customers=df_customers.rename(columns={'customer_zip_code_prefix':'geolocation_zip_code_prefix'})
df_customers.sample(3)

,customer_id,customer_unique_id,geolocation_zip_code_prefix,customer_city,customer_state
72602,df9e5a16c6e9ff85d78ca7ecc93af46b,d4017905e4548317a2065065c8cdfad0,1307,sao paulo,SP
31816,ac0f463cf83ca59742bdb8a44a864619,e27b1ce1665ecee46658235b3b0e0c7d,87014,maringa,PR
86805,2c524b1989927b7eccfc590b30e2f174,15b44b3aaf9255a983365b2a175b4a65,79415,sonora,MS


### EXPORT: tutti i dataframe puliti nella cartella output

In [48]:
import os

# Crea la cartella output se non esiste
os.makedirs("output", exist_ok=True)

dataframes_to_export = {
    "list_product":        df_list_product,
    "orders_items":        df_orders_items,
    "order_dataset":       df_order_dataset,
    "order_review":        df_order_review,
    "order_payments":      df_order_payments,
    "geolocation_dataset": df_geolocation_dataset,
    "list_seller":         df_list_seller,
    "customers":           df_customers,
}

for name, df in dataframes_to_export.items():
    df.to_csv(f"output/{name}.csv", index=False)